In [12]:
import torch
import numpy as np
import math, csv, gzip, os, io, copy, threading, re
import pandas as pd
import copy
import time as timee
from contextlib import contextmanager
from concurrent.futures import ThreadPoolExecutor, as_completed
from multiprocessing import Pipe, Pool
from scipy import stats
from pathlib import Path

In [13]:
from features import time_gap2, time2, connector_duplicate2, area_change2, recall_rate2, energy_dispersion2, \
    mean_voc_time2, n_unique_persons2
# %%
import features as features

In [14]:
@contextmanager
def timer(title):
    t0 = timee.time()
    yield
    print(f"{title} - done in {timee.time()-t0}s")

In [15]:
def normalize(mx:np.ndarray):  # 归一化
    shape=mx.shape
    mx=mx.reshape((-1,shape[-1]))
    for k in range(mx.shape[-1]):
        mx[:,k]=(mx[:,k]-np.min(mx[:,k]))/(np.max(mx[:,k])-np.min(mx[:,k]))
    mx=mx.reshape(shape)
    return mx

# 读数据、数据清洗

In [16]:
data_path = '~/pythonprojs/sichuan/'
all_user = pd.read_csv(data_path+'data/train_user.csv')
train_voc = pd.read_csv(data_path + 'data/train.csv')
test_voc = pd.read_csv(data_path + 'data/test.csv')
train_voc['start_datetime'] = pd.to_datetime(train_voc['start_datetime']) # 转换时间类型
test_voc['start_datetime'] = pd.to_datetime(test_voc['start_datetime']) # 转换时间类型
train_voc['hour'] = train_voc['start_datetime'].dt.hour  # 提取小时属性
test_voc['hour'] = test_voc['start_datetime'].dt.hour
train_voc = train_voc.merge(all_user[['phone_no_m', 'label']], how='left', on='phone_no_m') # 测试集合通话记录加上
test_voc = test_voc.merge(all_user[['phone_no_m', 'label']], how='left', on='phone_no_m')
all_user.set_index(keys='phone_no_m', drop=False, inplace=True)
train_voc=train_voc.set_index('phone_no_m',drop=False)
test_voc=test_voc.set_index('phone_no_m',drop=False)

In [17]:
train_user = set(train_voc['phone_no_m'].tolist()) # train_user set
test_user = set(test_voc['phone_no_m'].tolist())

# 先计算一部分特征

In [18]:

# calculate feature

# train_voc['mean_dur'] = train_voc.groupby(['phone_no_m', pd.Grouper(key='start_datetime', freq='W')])[
#     'call_dur'].transform(np.nanmean)  # 通话时长平均
# train_voc['var_dur'] = train_voc.groupby(['phone_no_m', pd.Grouper(key='start_datetime', freq='W')])[
#     'call_dur'].transform(np.nanvar)  # 通话时长方差
train_voc['sum_call_times'] = train_voc.groupby(['phone_no_m'])['phone_no_m'].transform('count')  #
train_voc['every_one_calltimes'] = train_voc.groupby(['phone_no_m', 'opposite_no_m'])['phone_no_m'].transform(
    'count')
train_voc['energy_dispersion'] = train_voc['every_one_calltimes'] / train_voc['sum_call_times']  # 精力分散度
# 入度/出度
train_voc['degree']=train_voc.groupby(['phone_no_m',pd.Grouper(key='start_datetime',freq='W'),'calltype_id'])['phone_no_m'].transform('count')
#邻居度
train_voc['neighbor_degree']=train_voc.groupby(['opposite_no_m',pd.Grouper(key='start_datetime',freq='W'),'calltype_id'])['phone_no_m'].transform('count')
del train_voc['sum_call_times']
del train_voc['every_one_calltimes']

# test_voc['mean_dur'] = test_voc.groupby(['phone_no_m', pd.Grouper(key='start_datetime', freq='W')])[
#     'call_dur'].transform(np.nanmean)  # 通话时长平均
# test_voc['var_dur'] = test_voc.groupby(['phone_no_m', pd.Grouper(key='start_datetime', freq='W')])[
#     'call_dur'].transform(np.nanvar)  # 通话时长方差
test_voc['sum_call_times'] = test_voc.groupby(['phone_no_m'])['phone_no_m'].transform('count')  #
test_voc['every_one_calltimes'] = test_voc.groupby(['phone_no_m', 'opposite_no_m'])['phone_no_m'].transform(
    'count')
test_voc['energy_dispersion'] = test_voc['every_one_calltimes'] / test_voc['sum_call_times']  # 精力分散度
# 入度/出度
test_voc['degree']=test_voc.groupby(['phone_no_m',pd.Grouper(key='start_datetime',freq='W'),'calltype_id'])['phone_no_m'].transform('count')
#邻居度
test_voc['neighbor_degree']=test_voc.groupby(['opposite_no_m',pd.Grouper(key='start_datetime',freq='W'),'calltype_id'])['phone_no_m'].transform('count')
del test_voc['sum_call_times']
del test_voc['every_one_calltimes']

# 构图

In [19]:
class Vertex:
    def __init__(self,key):
        self.id=key
        self.connect_to={}
    def add_neighbor(self,nbr,weight):
        self.connect_to[nbr]=weight
    def __str__(self):
        return str(self.id)
    def getId(self):
        return self.id
    def getNeighbors(self):
        return self.connect_to
class Graph:
    def __init__(self):
        self.verList=dict()
        self.numVertices=0
    def addVertex(self,key):
        if key not in self.verList:
            self.numVertices+=1
            self.verList[key]=Vertex(key)
    def addEdge(self,f,t,weight):
        self.addVertex(f)
        self.addVertex(t)
        self.verList[f].add_neighbor(t,weight)
        self.verList[t].add_neighbor(f,-weight)
    def __iter__(self):
        return iter(self.verList.values())
    def indegree(self,vertex):
        value=0
        for neigh,weight in self.verList[vertex].getNeighbors().items():
            if weight==-1:value+=1
        return value
    def outdegree(self,vertex):
        value=0
        for neigh,weight in self.verList[vertex].getNeighbors().items():
            if weight==1:value+=1
        return value
    def getCE(self,node):
        common_neigh=0
        for neighbor in self.verList[node].getNeighbors():
            for neighbor2 in self.verList[neighbor].getNeighbors():
                if neighbor2 in self.verList[node].getNeighbors():common_neigh+=1
        return common_neigh
    def getNeighbor(self,node):
        return self.verList[node].getNeighbors()

In [20]:
train_vocs = [g for _, g in train_voc.groupby(pd.Grouper(key='start_datetime', freq='W'))]
test_vocs = [g for _, g in test_voc.groupby(pd.Grouper(key='start_datetime', freq='W'))]

train_nets = [Graph() for i in range(len(train_vocs))]  # 训练集网络
test_nets = [Graph() for i in range(len(test_vocs))]  # 测试集网络
train_neighbor = {}  # neighbor of training set
test_neighbor = {}

for idx, net in enumerate(train_nets):  # 每个时间片构一个图
    for node in train_user:
        net.addVertex(node)
    voc = train_vocs[idx]
    for row in voc.itertuples():  # 每次通话
        calltype_id = getattr(row, 'calltype_id')
        source = getattr(row, 'phone_no_m')
        target = getattr(row, 'opposite_no_m')
        if calltype_id==1:
            net.addEdge(source,target,1)
            
        elif calltype_id==2:
            net.addEdge(source,target,-1)
            # net.addEdge(target,source,1)
        if source not in train_neighbor: train_neighbor[source] = []
        train_neighbor[source].append(target)

for idx, net in enumerate(test_nets):  # 每个时间片的图
    for node in test_user:
        net.addVertex(node)
    voc = test_vocs[idx]
    for row in voc.itertuples():  # 每次通话
        calltype_id = getattr(row, 'calltype_id')
        source = getattr(row, 'phone_no_m')
        target = getattr(row, 'opposite_no_m')
#         net.add_edge(source, target) if calltype_id == 1 else net.add_edge(target, source)
        if calltype_id==1:
            net.addEdge(source,target,1)
            net.addEdge(target,source,-1)
        elif calltype_id==2:
            net.addEdge(source,target,-1)
            net.addEdge(target,source,1)
        if source not in test_neighbor: test_neighbor[source] = []
        test_neighbor[source].append(target)


# 计算图特征和其余的动态特征

## 训练集

In [21]:
tmp_train_user = train_user.copy()
tmp_test_user = test_user.copy()

train_slice_feature = dict()
test_slice_feature = dict()

train_id_feature = dict()
test_id_feature = dict()

each_feature_costtime=[]

with timer('train feature '):
    # 计算动态特征
    for idx, slice in enumerate(train_vocs):  
        ps = [g for _, g in slice.groupby('phone_no_m')]
        net = train_nets[idx]
        
        for p in ps:  # each person in this slice
            p_dict=p.to_dict(orient='list')
            id = p['phone_no_m'].iloc[0]
            # 联系人重复率
            t0=timee.time()
            if idx == 0:
                repeat_rate = 0
            else:
                pre_week = train_vocs[idx - 1]
                pre_week = pre_week.loc[pre_week['phone_no_m'] == id]
                repeat_rate = connector_duplicate2(pre_week.to_dict(orient='list'), p_dict)
            t1=timee.time()
            if idx==0:each_feature_costtime.append(t1-t0)
            # 入度
            indegree = net.indegree(id)
            t2=timee.time()
            if idx==0:each_feature_costtime.append(t2-t1)
            # 出度
            outdegree = net.outdegree(id)
            t3=timee.time()
            if idx==0:each_feature_costtime.append(t3-t2)
            # 邻居平均度
            neighbor_degree = list()
            for neigh in net.getNeighbor(id):
                neighbor_degree.append(net.indegree(neigh)+net.outdegree(neigh))
            neighbor_degree = np.mean(neighbor_degree)
            t4=timee.time()
            if idx==0:each_feature_costtime.append(t4-t3)
            # 聚类系数
            coefficient = net.getCE(id)
            t5=timee.time()
            if idx==0:each_feature_costtime.append(t5-t4)
            # 回拨率
            recall_rate = features.recall_rate2(p_dict)
            t6=timee.time()
            if idx==0:each_feature_costtime.append(t6-t5)
            # 通话时间分布
            time_dis = [0 for i in range(24)]
            for time, count in p['hour'].value_counts(normalize=True).to_dict().items():
                time_dis[time] = count
            t7=timee.time()
            if idx==0:each_feature_costtime.append(t7-t6)
            # 平均通话时长
            mean_dur = p['mean_dur'].iloc[0]
            # 通话时长方差
            var_dur = p['var_dur'].iloc[0]
            if idx == 0: train_slice_feature[id] = list()
            train_slice_feature[id].append([indegree, outdegree, neighbor_degree, coefficient, recall_rate, repeat_rate, mean_dur,var_dur] + time_dis)
            tmp_train_user.discard(id)
        for id in list(tmp_train_user):  # persons not in this slice
            if idx == 0: train_slice_feature[id] = list()
            train_slice_feature[id] .append( [0 for i in range(32)])
        tmp_train_user=train_user.copy()
    print(np.array(each_feature_costtime)*len(train_vocs))
    # 计算身份特征
    static_voc = train_voc[['phone_no_m', 'energy_dispersion']].drop_duplicates(subset='phone_no_m').set_index(
        keys='phone_no_m')
    for id in list(train_user):
        idcard_cnt = all_user.loc[id, 'idcard_cnt']
        ed = static_voc.loc[id, 'energy_dispersion']
        train_id_feature[id] = [idcard_cnt, ed]
np_feature=[]   
np_id_feature=[]
labels=[]
for id, v in train_slice_feature.items(): 
    np_feature.append(v)
    np_id_feature.append(train_id_feature[id])
    labels.append(all_user.loc[id, 'label'])

np_feature = np.array(np_feature).astype(np.float)
np_id_feature = np.array(np_id_feature).astype(np.float)
labels = np.array(labels)
# train_n=int(len(np_feature)/4)*3
np_feature[:, :,[0,1, 2, 3, 6,7]] = normalize(np_feature[:, :,[0,1, 2, 3, 6, 7]])

np_id_feature[:,[0]]=normalize(np_id_feature[:,[0]])

np.save(arr=np_feature, file=data_path + f'data_after/mymodel/train_x.npy')
np.save(arr=np_id_feature, file=data_path + 'data_after/mymodel/train_x_id.npy')
np.save(arr=labels, file=data_path + f'data_after/mymodel/train_y.npy')


d:\software\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\software\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
d:\software\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\software\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
d:\software\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\software\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
d:\software\anaconda3\lib

[0.         0.         0.         ... 0.         0.         0.02917385]
train feature  - done in 434.8685438632965s


C:\Users\82584\AppData\Local\Temp\ipykernel_16964\2283971256.py:87: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np_feature = np.array(np_feature).astype(np.float)
C:\Users\82584\AppData\Local\Temp\ipykernel_16964\2283971256.py:88: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np_id_feature = np.array(np_id_feature).astype(np.float)


## 测试集

In [24]:
with timer('test feature '):
    # 计算动态特征
    for idx, slice in enumerate(test_vocs):  
        ps = [g for _, g in slice.groupby('phone_no_m')]
        net = test_nets[idx]
        # net: nx.DiGraph
        for p in ps:  # each person in this slice
            p_dict=p.to_dict(orient='list')
            id = p['phone_no_m'].iloc[0]
            if idx == 0:
                repeat_rate = 0
            else:
                pre_week = test_vocs[idx - 1]
                pre_week = pre_week.loc[pre_week['phone_no_m'] == id]
                repeat_rate = connector_duplicate2(pre_week.to_dict(orient='list'), p_dict)
            indegree = net.indegree(id)
            outdegree = net.outdegree(id)
            neighbor_degree = list()
            for neigh in net.getNeighbor(id):
                neighbor_degree.append(net.indegree(neigh)+net.outdegree(neigh))
            neighbor_degree = np.mean(neighbor_degree)
            coefficient = net.getCE(id)
            recall_rate = features.recall_rate2(p_dict)
            time_dis = [0 for i in range(24)]
            for time, count in p['hour'].value_counts(normalize=True).to_dict().items():
                time_dis[time] = count
            
            mean_dur = p['mean_dur'].iloc[0]
            var_dur = p['var_dur'].iloc[0]
            if idx == 0: test_slice_feature[id] = list()
            test_slice_feature[id].append([indegree, outdegree, neighbor_degree, coefficient, recall_rate, repeat_rate, mean_dur,var_dur] + time_dis)
            tmp_test_user.discard(id)
        for id in list(tmp_test_user):  # persons not in this slice
            if idx == 0: test_slice_feature[id] = list()
            test_slice_feature[id] .append( [0 for i in range(32)])
        tmp_test_user=test_user.copy()
    # 计算身份特征
    static_voc = test_voc[['phone_no_m', 'energy_dispersion']].drop_duplicates(subset='phone_no_m').set_index(
        keys='phone_no_m')
    for id in list(test_user):
        idcard_cnt = all_user.loc[id, 'idcard_cnt']
        ed = static_voc.loc[id, 'energy_dispersion']
        test_id_feature[id] = [idcard_cnt, ed]
np_feature=[]   
np_id_feature=[]
labels=[]
for id, v in test_slice_feature.items(): 
    np_feature.append(v)
    np_id_feature.append(test_id_feature[id])
    labels.append(all_user.loc[id, 'label'])

np_feature = np.array(np_feature).astype(np.float)
np_id_feature = np.array(np_id_feature).astype(np.float)
labels = np.array(labels)
# test_n=int(len(np_feature)/4)*3
np_feature[:, :,[0,1, 2, 3, 6,7]] = normalize(np_feature[:, :,[0,1, 2, 3, 6, 7]])

np_id_feature[:,[0]]=normalize(np_id_feature[:,[0]])

np.save(arr=np_feature, file=data_path + f'data_after/mymodel/test_x.npy')
np.save(arr=np_id_feature, file=data_path + 'data_after/mymodel/test_x_id.npy')
np.save(arr=labels, file=data_path + f'data_after/mymodel/test_y.npy')

d:\software\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\software\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
d:\software\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\software\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
d:\software\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\software\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
d:\software\anaconda3\lib

test feature  - done in 112.36120414733887s


C:\Users\82584\AppData\Local\Temp\ipykernel_16964\4109290276.py:52: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np_feature = np.array(np_feature).astype(np.float)
C:\Users\82584\AppData\Local\Temp\ipykernel_16964\4109290276.py:53: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np_id_feature = np.array(np_id_feature).astype(np.float)


In [25]:
train_x=np.load('D:\\pycharmProjs\\sichuan_data\\data_after\\mymodel\\train_x.npy')
test_x=np.load('D:\\pycharmProjs\\sichuan_data\\data_after\\mymodel\\test_x.npy')

In [26]:
train_x=np.nan_to_num(train_x,)
test_x=np.nan_to_num(test_x,)

In [ ]:
np.save('D:\\pycharmProjs\\sichuan_data\\data_after\\mymodel\\train_x.npy',train_x)
np.save('D:\\pycharmProjs\\sichuan_data\\data_after\\mymodel\\test_x.npy',test_x)